In [1]:
import pandas as pd
product_data = pd.read_excel(r"E:\NIIT\Project_Files\Datasets\product_data.xlsx",usecols=['Category', 'Price', 'Product Name', 'Product Description', 'Product Details',
                                                                                          'Brand'],nrows=1000)
product_data.head()

,Category,Price,Product Name,Product Description,Product Details,Brand
0,Electronics,349,Mi Case for Redmi 4A (Black),"Extremely light and thin,Compatible Models: Re...",Brand Mi; Model 4A; Model Name 4A; Item Weight...,Mi
1,Electronics,0,"OnePlus 3T (Soft Gold, 64GB)(Certified Refurbi...",This Certified Refurbished product is tested a...,OS Android; RAM 6 GB; Item Weight 159 g; Produ...,OnePlus
2,Computer Accessories,1770,HP Compaq MU06 Laptop Original Battery with La...,"No. Of Cells : 6,Capacity (Mah) : 4400,Voltage...","No. Of Cells : 6,Capacity (Mah) : 4400,Voltage...",NaN
3,Laptops,66999,Dell 15R 5537 Laptop (4th Gen Intel Core i7 45...,"39.62cm (15.6) Touchscreen,8 GB RAM,Fully Engl...","39.62cm (15.6) Touchscreen,8 GB RAM,Fully Engl...",NaN
4,Smart Phones,47470,Apple iPhone 7 32 GB (Black),Screen Size : 11.93 cm (4.7)Camera: 12 MP Rear...,Screen Size : 11.93 cm (4.7)Camera: 12 MP Rear...,Apple


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
import gradio as gr
import requests
from PIL import Image
from io import BytesIO
import random
from textwrap import dedent
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from scipy import sparse

# Define sentiment analysis function
sia = SentimentIntensityAnalyzer()


def sentiment_analysis(text):
    result = sia.polarity_scores(str(text))
    score = result['compound']
    if score > 0.1:
        sentiment = 'Positive'
        response_image = requests.get('https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQq3rG7aRuKd8zjf70GxeOtz1ApuGYB9beW7uVOvu17NQ&usqp=CAU&ec=48665699.jpg')
    elif score < -0.1:
        sentiment = 'Negative'
        response_image = requests.get('https://belvoir-university-health.s3.amazonaws.com/media/2018/11/08152705/how-to-deal-with-negative-people-607x552.jpg')
    else:
        sentiment = 'Neutral'
        response_image = requests.get('https://elements-cover-images-0.imgix.net/7cab172f-01c1-4774-a151-7bcf11a8c6b9?auto=compress%2Cformat&fit=max&w=1019&s=117179cbde33d291d072a096b466a28d.jpg')
    output_text = f"The sentiment analysis of your text: {sentiment}.\nThank you for using my model."
    output_image = Image.open(BytesIO(response_image.content))
    return output_text, output_image


interface1=gr.Interface(
    fn=sentiment_analysis,
    inputs=gr.inputs.Textbox(),
    outputs=[gr.outputs.Textbox(), gr.outputs.Image(type='pil')],
    examples=[
        ['I love this product'],
        ['This is terrible.']
    ],
    title='Sentiment Analysis',
    description='A deep learning model to analyze the sentiment of a text.',
    theme='compact'
)


# Create TF-IDF vectorizer
# This line creates an instance of the TfidfVectorizer class, which is a tool for converting text data into a numerical format
tfidf = TfidfVectorizer(stop_words='english')

# Fit and transform the text data
# transforms the data into a numerical format using the TF-IDF algorithm. 
# The astype(str) method is used to ensure that the data is in string format before passing it to the vectorizer.
tfidf_matrix = tfidf.fit_transform(product_data['Product Description'].astype(str))

# Convert the sparse matrix to a compressed sparse row (CSR) matrix
# This line converts the output of the TF-IDF vectorizer to a compressed sparse row  matrix format
tfidf_matrix = sparse.csr_matrix(tfidf_matrix)

# Compute cosine similarity using linear kernel
# This line computes the cosine similarity between all pairs of products in the dataset using the linear kernel method.
# Cosine similarity is a measure of similarity between two vectors,and the linear kernel method is a way to compute this similarity efficiently.
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Create a Series mapping product name to index
indices = pd.Series(product_data.index, index=product_data['Product Name']).drop_duplicates()

def get_recommendation(title):
    # checks if the input product name is in the indices Series, and returns an error message if it is not found.
    if title not in indices:
        return "Title not found in dataset"
    # the function looks up the index of the product in the product_data DataFrame using the indices Series.
    idx = indices[title]
    # similarity scores between the input product and all other products using the cosine similarity matrix.
    sim_scores = list(enumerate(cosine_sim[idx]))
    # sorted() function is called on the sim_scores list
    # key=lambda x: x[1] specifies that the second element of each of the similarity score is used as the key for sorting.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    #top 10 most similar products (excluding the input product itself) are selected as the recommendations.
    sim_scores = sim_scores[1:11]
    #  creates a list sim_index containing the first element  of each tuple in sim_scores. This list contains 
    # the indices of the products that are most similar to the input product.
    sim_index = [i[0] for i in sim_scores]
    # selects the names of the products that correspond to the indices in sim_index from the original product_data DataFrame. 
    #These are the names of the products that are most similar to the input product.
    recommendations = product_data['Product Name'].iloc[sim_index]
    return "\n".join(recommendations)


interface2=gr.Interface(
    fn=get_recommendation,
    inputs=gr.inputs.Textbox(label='Product Name', type='text'),
    outputs=gr.outputs.Textbox(label='Recommended Products', type='text'),
    title='<b>PRODUCT RECOMMENDER</b>',
    description='<b>Enter a product name and get a list of recommended products.</b>',theme='compact',
    examples=[['Mi Case for Redmi 4A (Black)'], ['Bombay Dyeing Cotton Floral Double BedsheetÂ\xa0Â\xa0(1 double bedsheet with 2 pillow cover, Purple)'],['iBall High Speed Card Reader MCR20'],['Thick & Full Biotin & Collagen Conditoner 385 ml']],
    )

interface_list = [interface1, interface2]
tabbed_interface = gr.TabbedInterface(interface_list,
                                      ["Sentiment Analysis", "Product Recommender"],
                                      title="NIIT",
                                      css = ".gradio-container {background: url('file=https://images.pexels.com/photos/4087576/pexels-photo-4087576.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=2.png');background-size: cover;}"
                                      )

tabbed_interface.launch(share=True, auth=('team3', 'dsft13'), auth_message="Hello NIIT Learner, please provide your credentials.")


Running on local URL:  http://127.0.0.1:7864
Running on public URL: https://ab54021fe00ec91545.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


In [10]:
indices

Product Name
Mi Case for Redmi 4A (Black)                                                                              0
OnePlus 3T (Soft Gold, 64GB)(Certified Refurbished)                                                       1
HP Compaq MU06 Laptop Original Battery with Lapronics Keyboard Protector - 3 Years Warranty               2
Dell 15R 5537 Laptop (4th Gen Intel Core i7 4500U- 8GB RAM- 1TB HDD- 39.62cm (15.6) Touchscreen- Win      3
 Apple iPhone 7 32 GB (Black)                                                                             4
                                                                                                       ... 
Thick & Full Biotin & Collagen Conditoner 385 ml                                                        995
Casio EX206 Edifice Analog Watch - For Men                                                              996
Healthgenie ICE BAG used for First Aid, Sports Injury, Pain Relief, Cold Therapy                        997
iBall High Spee